# Part I Write a Python program based on the provided Class Exercise, which:

#### a) Loads the downloaded pre-trained Google Word2Vec model from your computer

In [ ]:
### load Word2Vec model

import gensim, operator
from scipy import spatial
import numpy as np
from gensim.models import KeyedVectors

model_path = '/Users/arlenehuang/OneDrive - Columbia University/Summer 2020/APAN 5430/Assignment 6/'

def load_wordvec_model(modelName, modelFile, flagBin):
    print('Loading ' + modelName + ' model...')
    model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin)
    print('Finished loading ' + modelName + ' model...')
    return model

model_word2vec = load_wordvec_model('Word2Vec', 'GoogleNews-vectors-negative300.bin.gz', True)

Loading Word2Vec model...
Finished loading Word2Vec model...


#### b) Loads your previously obtained dataset of Webhose news articles

In [ ]:
### read webhose data into newsfeeds
import json
json_data = open("webhose_Citigroup.json").readlines()
newsfeeds = []
for lines in json_data:
    newsfeeds.append(json.loads(lines))

In [ ]:
newsfeeds[1]

{'thread': {'uuid': '4fbfdd930f5a0684da8f0b37ca8a364cda02c716',
  'url': 'https://rocklandregister.com/2020/06/13/esperion-therapeutics-espr-buy-rating-reaffirmed-at-credit-suisse-group.html',
  'site_full': 'rocklandregister.com',
  'site': 'rocklandregister.com',
  'site_section': 'https://rocklandregister.com/rss',
  'site_categories': ['media'],
  'section_title': 'Rockland Register',
  'title': 'Esperion Therapeutics’ (ESPR) Buy Rating Reaffirmed at Credit Suisse Group',
  'title_full': 'Esperion Therapeutics’ (ESPR) Buy Rating Reaffirmed at Credit Suisse Group',
  'published': '2020-06-13T16:36:00.000+03:00',
  'replies_count': 0,
  'participants_count': 1,
  'site_type': 'news',
  'country': 'US',
  'spam_score': 0.0,
  'main_image': 'https://www.marketbeat.com/logos/esperion-therapeutics-inc-logo.png',
  'performance_score': 0,
  'domain_rank': None,
  'social': {'facebook': {'likes': 0, 'comments': 0, 'shares': 0},
   'gplus': {'shares': 0},
   'pinterest': {'shares': 0},
   '

#### c) For any one selected article title from the dataset, finds 100 most similar titles based on Word2Vec similarity, prints those titles in the reverse order of similarity scores

In [ ]:
# define a function

def vec_similarity(input1, input2, vectors):
    term_vectors = [np.zeros(300), np.zeros(300)]
    terms = [input1, input2]
        
    for index, term in enumerate(terms):
        for i, t in enumerate(term.split(' ')):
            try:
                term_vectors[index] += vectors[t]
            except:
                term_vectors[index] += 0
        
    result = (1 - spatial.distance.cosine(term_vectors[0], term_vectors[1]))
    if result == 'nan':
        result = 0
        
    return result

def vocab_check(vectors, words):
    
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

def calc_similarity(input1, input2, vectors):
    s1words = set(vocab_check(vectors, input1.split()))
    s2words = set(vocab_check(vectors, input2.split()))
    
    output = vectors.n_similarity(s1words, s2words)
    return output

def get_100_similar_titles(dataset, num_of_target_article):
    similarities = []
    title = dataset[num_of_target_article]['title']
    
    for i in range(len(dataset)):
        news_title = dataset[i]['title']
        similarity = calc_similarity(title, news_title, model_word2vec)
        similarities.append(similarity)
    
    import heapq
    max_num_index = map(similarities.index, heapq.nlargest(101,similarities))

    print('\033[1m' + '\033[4m' + 'The 100 Most Similar Titles Of Article: ', title + '\033[0m')
    num = 0
    for j in max_num_index:
        if similarities[j] != 1 :
            num += 1
            print('No.', num)
            print('title: ', newsfeeds[j]['title'])
            print('similarity: ', similarities[j])

In [ ]:
# apply it
get_100_similar_titles(newsfeeds, 188)

The 100 Most Similar Titles Of Article:  Neurocrine Biosciences (NASDAQ:NBIX) Earns Outperform Rating from Analysts at Wedbush
No. 1
title:  Intercept Pharmaceuticals (NASDAQ:ICPT) Earns Buy Rating from Wedbush
similarity:  0.83997256
No. 2
title:  Wedbush Reiterates Buy Rating for BioMarin Pharmaceutical (NASDAQ:BMRN)
similarity:  0.8227078
No. 3
title:  Despegar.com (NYSE:DESP) Earns Buy Rating from Analysts at Citigroup
similarity:  0.8179383
No. 4
title:  Despegar.com (NYSE:DESP) Earns Buy Rating from Analysts at Citigroup
similarity:  0.8179383
No. 5
title:  Globant (NYSE:GLOB) Earns Overweight Rating from Analysts at Piper Sandler
similarity:  0.81530195
No. 6
title:  BioMarin Pharmaceutical’s (BMRN) Buy Rating Reiterated at Wedbush
similarity:  0.81173915
No. 7
title:  Zoetis (NYSE:ZTS) Earns Buy Rating from Analysts at Jefferies Financial Group
similarity:  0.8091602
No. 8
title:  UnitedHealth Group (NYSE:UNH) Earns Buy Rating from Analysts at Goldman Sachs Group
similarity:  0

# Part II Write a Pyspark program based on the other provided Class Exercise, which:

#### a) Loads your previously obtained dataset of Webhose news articles into a Spark dataframe

In [2]:
#pip install pyspark

In [3]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext() 
sqlContext = SQLContext(sc)
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.mllib.clustering import LDA, LDAModel
from nltk.stem.wordnet import WordNetLemmatizer
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, Word2Vec

print("Using Apache Spark Version", sc.version)

Using Apache Spark Version 3.0.0


In [5]:
crunchbase_df = sqlContext.read.option("header", "true").option("delimiter", ",") \
                    .option("inferSchema", "true") \
                    .json("/Users/arlenehuang/OneDrive - Columbia University/Summer 2020/APAN 5430/Assignment 6/webhose_Citigroup.json")
crunchbase_df.columns

['author',
 'crawled',
 'entities',
 'external_images',
 'external_links',
 'highlightText',
 'highlightThreadTitle',
 'highlightTitle',
 'language',
 'ord_in_thread',
 'parent_url',
 'published',
 'rating',
 'text',
 'thread',
 'title',
 'updated',
 'url',
 'uuid']

In [6]:
crunchbase_data = crunchbase_df['uuid','title','text']

In [7]:
crunchbase_columns = [0,1,2]
crunchbase_rdd = crunchbase_data.select('*') \
                       .rdd.map(lambda row: [row[i] for i in crunchbase_columns]) \
                       .filter(lambda row: row[2] is not None)
crunchbase_df = sqlContext.createDataFrame(crunchbase_rdd, 
                                           ['uuid','title','text'])
crunchbase_df.show(5)

+--------------------+--------------------+--------------------+
|                uuid|               title|                text|
+--------------------+--------------------+--------------------+
|efe37c1b78ac4e480...|It’s like the Wil...|Tales of out-of-w...|
|4fbfdd930f5a0684d...|Esperion Therapeu...|Credit Suisse Gro...|
|6fc2347180fc2ba32...|Why Citi is the l...|The Members Excha...|
|b61d35ac80ccbb587...|Centene (NYSE:CNC...|Centene (NYSE:CNC...|
|1b07dc8633ab2ce32...|CVS Health (NYSE:...|CVS Health (NYSE:...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



#### b) Cleans up and tokenizes article bodies using the RegexTokenizer and Stopword remover functions provided in the Class Exercise

In [8]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load( disable=['parser', 'tagger','ner'] )

def cleanup_pretokenize(text):
    #text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'http\S+', '', text)
    text = text.replace("'s", " ")
    text = text.replace("n't", " not ")
    text = text.replace("'ve", " have ")
    text = text.replace("'re", " are ")
    text = text.replace("I'm"," I am ")
    text = text.replace("you're"," you are ")
    text = text.replace("You're"," You are ")
    text = text.replace("-"," ")
    text = text.replace("/"," ")
    text = text.replace("("," ")
    text = text.replace(")"," ")
    text = text.replace("%"," percent ")
    return text

lmtzr = WordNetLemmatizer()
def text_cleanup(row):
    desc = row[2].strip().lower()
    tokens = [w.lemma_ for w in nlp(cleanup_pretokenize(desc))]
    tokens = [token for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if len(token) > 3]
    #tokens = [lmtzr.lemmatize(token,'v') for token in tokens]
    row[2] = ' '.join(tokens)
    return row

regexTokenizer = RegexTokenizer(gaps = False, pattern = '\w+', inputCol = 'text', outputCol = 'tokens')
swr = StopWordsRemover(inputCol = 'tokens', outputCol = 'tokens_sw_removed')

In [9]:
df_tokens = regexTokenizer.transform(crunchbase_df)
desc_swr = swr.transform(df_tokens)
desc_swr.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                uuid|               title|                text|              tokens|   tokens_sw_removed|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|efe37c1b78ac4e480...|It’s like the Wil...|Tales of out-of-w...|[tales, of, out, ...|[tales, work, 20,...|
|4fbfdd930f5a0684d...|Esperion Therapeu...|Credit Suisse Gro...|[credit, suisse, ...|[credit, suisse, ...|
|6fc2347180fc2ba32...|Why Citi is the l...|The Members Excha...|[the, members, ex...|[members, exchang...|
+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



#### c) Trains a Word2Vec model based on the output column produced in step 2

In [10]:
word2vec = Word2Vec(vectorSize = 300, minCount = 5, inputCol = 'tokens_sw_removed', outputCol = 'wordvectors')
model = word2vec.fit(desc_swr)
wordvectors = model.transform(desc_swr)
crunchbase_desc = wordvectors.select('uuid','text','wordvectors').rdd.toDF()
crunchbase_desc.show(5)

+--------------------+--------------------+--------------------+
|                uuid|                text|         wordvectors|
+--------------------+--------------------+--------------------+
|efe37c1b78ac4e480...|Tales of out-of-w...|[0.04399570427517...|
|4fbfdd930f5a0684d...|Credit Suisse Gro...|[0.03478927901529...|
|6fc2347180fc2ba32...|The Members Excha...|[0.06360735781774...|
|b61d35ac80ccbb587...|Centene (NYSE:CNC...|[0.03624564075376...|
|1b07dc8633ab2ce32...|CVS Health (NYSE:...|[0.04306173842644...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [11]:
synonyms = model.findSynonyms("facebook", 20)   
synonyms.show()

+---------+------------------+
|     word|        similarity|
+---------+------------------+
|  twitter|0.9272628426551819|
|   google|0.8422284722328186|
|    south|0.8214764595031738|
|    named|0.8141477108001709|
|  protect|0.8031668663024902|
| atlantic| 0.795950174331665|
|  respond|0.7940086126327515|
|   taking|0.7912166118621826|
|  housing|0.7908703684806824|
|    doesn|0.7855764031410217|
|       mr|0.7767440676689148|
|recycling|0.7756271362304688|
| domestic|0.7734577059745789|
| approach|   0.7730712890625|
|   center|0.7693139314651489|
|      rid|0.7688893675804138|
|     best|0.7673935890197754|
|      bet|0.7669375538825989|
|   rights|0.7655760645866394|
|  purpose| 0.763039767742157|
+---------+------------------+



#### d) Implements any sample search query, as shown in Class Exercise, and produces matching article titles

In [13]:
chunk = crunchbase_desc.take(50000)

def cossim(v1, v2): 
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / (np.sqrt(np.dot(v2, v2))+.1)

Version 1: same as class excise

In [14]:
SEARCH_QUERY = "CVS Health"

In [15]:
query_df  = sc.parallelize([(1,SEARCH_QUERY)]).toDF(['index','text'])
query_tok = regexTokenizer.transform(query_df)
query_swr = swr.transform(query_tok)
query_swr.show()
query_vec = model.transform(query_swr)
query_vec = query_vec.select('wordvectors').collect()[0][0]

+-----+----------+-------------+-----------------+
|index|      text|       tokens|tokens_sw_removed|
+-----+----------+-------------+-----------------+
|    1|CVS Health|[cvs, health]|    [cvs, health]|
+-----+----------+-------------+-----------------+



In [16]:
import numpy as np

sim_rdd = sc.parallelize((i[0], i[1], float(cossim(query_vec, i[2]))) for i in chunk)
sim_df  = sqlContext.createDataFrame(sim_rdd).\
                   withColumnRenamed('_1', 'uuid').\
                   withColumnRenamed('_2', 'title').\
                   withColumnRenamed('_3', 'similarity').\
                   orderBy("similarity", ascending = False)
sim_df.show(20, truncate = True)

+--------------------+--------------------+-------------------+
|                uuid|               title|         similarity|
+--------------------+--------------------+-------------------+
|53c96fd686936e216...|$3.90 4.72
Citigr...| 0.4664175021366124|
|96dbeb28b6e17b625...|Retail Banking Ma...|0.45110269133003483|
|ac1b5727abae0287e...|Neobanking Market...| 0.4451130883259087|
|5f0a416f899b9d88e...|NEW YORK and TEL ...| 0.4108399286968714|
|857653dcb5879692b...|By Nisha Gopalan
...|0.37709951763493144|
|15488d0b142da1897...|I returned to the...| 0.3697382785591337|
|cd7255b75a7286733...|The office isn't ...| 0.3677985453920586|
|0f1214735390c72e0...|by Maurício Angel...| 0.3646207685124295|
|1616b88e77f89cefe...|More
By NISHA GOP...| 0.3641083809090477|
|18b126621b8c6f143...|By A.R. Shaw | Ju...|0.35992534563836137|
|65555cdad7b11c897...|Banks including H...|0.35775898390072575|
|b24989ab7cc065ef4...|Difference betwee...|0.34633115365477807|
|bfaa716284689fc0c...|In a time of a gl.

Version 1: using input function

In [17]:
SEARCH_QUERY = input("search query: ")

query_df  = sc.parallelize([(1,SEARCH_QUERY)]).toDF(['index','text'])
query_tok = regexTokenizer.transform(query_df)
query_swr = swr.transform(query_tok)
query_vec = model.transform(query_swr)
query_vec = query_vec.select('wordvectors').collect()[0][0]

import numpy as np

sim_rdd = sc.parallelize((i[0], i[1], float(cossim(query_vec, i[2]))) for i in chunk)
sim_df  = sqlContext.createDataFrame(sim_rdd).\
                   withColumnRenamed('_1', 'uuid').\
                   withColumnRenamed('_2', 'title').\
                   withColumnRenamed('_3', 'similarity').\
                   orderBy("similarity", ascending = False)
sim_df.show(20, truncate = True)

search query: CVS Health
+--------------------+--------------------+-------------------+
|                uuid|               title|         similarity|
+--------------------+--------------------+-------------------+
|53c96fd686936e216...|$3.90 4.72
Citigr...| 0.4664175021366124|
|96dbeb28b6e17b625...|Retail Banking Ma...|0.45110269133003483|
|ac1b5727abae0287e...|Neobanking Market...| 0.4451130883259087|
|5f0a416f899b9d88e...|NEW YORK and TEL ...| 0.4108399286968714|
|857653dcb5879692b...|By Nisha Gopalan
...|0.37709951763493144|
|15488d0b142da1897...|I returned to the...| 0.3697382785591337|
|cd7255b75a7286733...|The office isn't ...| 0.3677985453920586|
|0f1214735390c72e0...|by Maurício Angel...| 0.3646207685124295|
|1616b88e77f89cefe...|More
By NISHA GOP...| 0.3641083809090477|
|18b126621b8c6f143...|By A.R. Shaw | Ju...|0.35992534563836137|
|65555cdad7b11c897...|Banks including H...|0.35775898390072575|
|b24989ab7cc065ef4...|Difference betwee...|0.34633115365477807|
|bfaa716284689f